### Full Range

策略： 在full range内提供流动性

In [6]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [7]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 10000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>


In [8]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = 5000
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = 5000 * price_begin_real

this is the cursor: 1682424000


### 计算该策略的手续费

首先需要计算出头寸的liquidity是多少。

full range策略相当于v2中的头寸，在全价格范围内提供流动性，在v3中就是minTick和maxTick(0,infinity)之间提供流动性。

我在测试网上提供了一个full range的[头寸](https://goerli.etherscan.io/tx/0x199a0a53afbaa7c6b542dc2bdc2766639e9250055032238ebcf204d7375c8aa5), 得到：

tickLower： -887270
tickUpper： 887270


In [9]:
# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)

this is the lower price:  2.939544628365392e-39
this is the upper price:  3.4018874568203963e+38


In [10]:
# 计算liquidity
l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
l = min(l1, l2)
print("l1:", l1)
print("l2:", l2)
print("l:", l)

l1: 2.710878376267272e-16
l2: 1.5128922860696844e-19
l: 1.5128922860696844e-19


In [11]:
# 获取选定时间期限内的所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 0x011c667d9faa12abe7bead6640eb2e3f70fbb0cc3b7658e53224b0e9cc7fbccc#4802536
this is the cursor: 0x0244fed2fe5c490934b4ae18cd559413b6ac11b41093934ba15dac2f5354ee78#4761962
this is the cursor: 0x035a2b8843092c3e6129766cfd98d94e59d5c9960cdf3d65f85bd980813c7b7d#4663593
this is the cursor: 0x0496a66b2160386dded5b2db30c1edec8e342edb48e0a9b9a2017e40d40771a0#4805900
this is the cursor: 0x05b839deb930323bf2b631e1c194d1ddef813773a98e46277e5ea327e1b11460#4764407
this is the cursor: 0x06eb10f0f7f11e5c4414ffbaa72b59248433bb353f2bdcc9d593e37048b38709#4733024
this is the cursor: 0x080b26914ce6682dc3f04a48d0b6e8946105952d580cfd65ecf71b936bbe0ce7#4824830
this is the cursor: 0x093213c88791935b6eb76fe957f9d7eaa3899042fa4520b741afd078c11b4e1c#4672231
this is the cursor: 0x0a5717e9aa1639d384baa599a265269d7f79693cb3cd3bd1c6a967cfae88d350#4752517
this is the cursor: 0x0b80ffc07af84dc8913802cee50cdd593e410885332c5c885cc52cb7dd9592a9#4721296
this is the cursor: 0x0ca4132d5f42e9ce8310d8919e1e

In [14]:
# 计算fee
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)
# transform X86 into normal number
swaps["sqrtPriceX96"] = (swaps["sqrtPriceX96"] / (2**96) ) ** 2
# transform sqrt_p into p
swaps["sqrtPriceX96"] = 1 / (swaps["sqrtPriceX96"] * (10 ** 12))

In [15]:
# 现在来加总然后计算手续费
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick
0,1679680919,-19922.526611,11.442382,2068644751680517294034222771845249752126548235...,201692
1,1680218951,1104.872406,-0.617085,1960012268168393855424962344838531227727494220...,201422
2,1681392455,-3951.498533,1.973163,1563623108622693998474154391442424616851718077...,200293
3,1679774651,-12033.180287,7.000000,2122123923112001198876444686643213644941502947...,201820
4,1681089035,3299.547442,-1.774376,1817079095083724900454340692570255881729117000...,201044
5,1679404235,-5000.197259,2.750358,1897291939700589651204306550317627778214583174...,201260


In [16]:
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 1682424000


In [17]:
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1679158800,31691854789759819627,1871666948661873416199899917784088,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,0,0,0,372,1798.390894813620679474180985197866,1803.236399827628569085607995221095,1769.953587075568576406795835267567,1791.851542392230339967547914533666
1,1679162400,32915185914729732691,1867893400320714844515651086096830,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,0,0,0,366,1790.640320160079804025064362917098,1799.108571408040728063168297875355,1786.686354081499257894742581324593,1799.098710137099104209582500255936
2,1679166000,34369179026466248085,1869482743679436927327266292647007,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,0,0,0,333,1799.099225334904656531075427168004,1809.622901299084675474781496124223,1794.120699231514675114167039962852,1796.040997899866322473028892703993
3,1679169600,34183163223601833767,1868781276195693273308669229878293,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,0,0,0,317,1796.054690522833853305120372567828,1805.57781837591635090288239389413,1792.725482994450340228488201862034,1797.389578204559120182790388692096
4,1679173200,34246952167731560476,1875752556320845172108591732547304,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,0,0,0,309,1797.199358810234823401563218695319,1797.220201287329445413990798860576,1783.038903387140753714413070004309,1784.054320332921814271944303424757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,1682409600,16112410314713274755,1855396075476526233522468623073711,1823.416583151871396365506083528169,0.0005484210296428517961065025005506124,201235,2354886033132284336509159773621094,1132563718699845738718890057150461336780111,454715802.1771811339316673731237985,0,0,0,0,219,1816.379226851717164100898127193097,1823.628340218834384804877731532306,1807.898200099231728631450494707573,1823.416583151871396365506083528169
904,1682413200,16137937062166665983,1860134945830249176006803422127704,1814.137763884581602539400031168837,0.0005512260534496109126405270701638359,201286,2354902274179600062148961039052437,1132583037334689801823590630015564305195314,454118366.3995681874113951251124435,0,0,0,0,159,1823.406923101170500016424867465148,1824.612589203133525389173051572799,1813.855242446223713175481937692827,1814.137763884581602539400031168837
905,1682416800,19013160413615431381,1859449813234947837954853577493161,1815.474884025731406836998768140546,0.0005508200685115215423289471983162687,201279,2354927677378531251825827032525679,1132595559083858227179230509791232204689709,454107808.6245183972102490081734885,0,0,0,0,177,1814.1696238717811877536741604071,1817.01437154306006210007788280662,1813.368815609048533449969620530264,1815.474884025731406836998768140546
906,1682420400,16167375475342206765,1853808722878671136176945468001502,1826.540577773547970576597060886161,0.0005474830464587568811983396798922428,201218,2354971492877263596435574642760511,1132607493696938787645699475557173878568155,454751549.2083887890337426200788898,0,0,0,0,277,1815.480349123526192318017836764214,1828.304667651261251579256442899603

In [18]:
liquidity_data = liquidity_data[["periodStartUnix", "liquidity"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "liquidity": "liquidity"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "liquidity"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,liquidity
0,0x0861d00acafcd74202e82741c031db7417fbca4c45fe...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,-17615.760520,9.835104,17613.789058,1954742770748450983934394729019639498989354272...,201409,17,31691854789759819627
1,0x060ca03d50d79c0196c3b46dcbebb3dfef605b4f815e...,1679155343,0xe592427a0aece92de3edee1f18e0157c05861564,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xbff9a1b539516f9e20c7b621163e676949959a66,5994.484682,-3.349238,5995.998881,1961455742721429099562709721131276229618426889...,201426,168,31691854789759819627
2,0x00a4333a3aad92b9579333e786936077d987a2022a6f...,1679155355,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xa62a7a20c38fd87d0ba3f06bfc84a3bce7596120,619.112400,-0.345987,619.337729,1962342469367604417141141135379594424033396105...,201428,299,31691854789759819627


In [19]:
# 获取头寸的liquidity，后续手续费的分成是按照头寸的liquidity占总liquidity的比例来计算的
position_liquidity = l
merged_data['liquidity'] = merged_data['liquidity'].astype(float)
merged_data['ratio'] = float(position_liquidity) / merged_data["liquidity"]
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)

2.3991317094188813e-33
1.3249882327646663e-36


*** 

以下是用invest metric来计算投资指标的内容

In [109]:
import sys
sys.path.append("..") 

from investMetric import metric
reload(metric)

# roi1 = metric.roi(fee1_sum + token1_amount,  token1_amount, (end - begin).days)
# print("** roi0 = ", f"x = {roi0[1]:.2f}")
# print("** roi1 = ", f"x = {roi1[1]:.2f}")

** roi0 =  x = 0.00
** roi1 =  x = 0.00


In [5]:
begin = datetime(2023, 3, 19)
end = datetime.now()
days = (end - begin).days
days

37

In [113]:
merged_data["position_value"].iloc[-1]

10000.616779756558

In [114]:
merged_data["fee0_sum"] = merged_data['fee0'].cumsum()
merged_data["fee1_sum"] = merged_data['fee1'].cumsum()
# transform X86 into normal number
merged_data["real_price"] = merged_data["tick"].apply(lambda x: 1/ (utils.price2RealPrice(utils.tickIndex2Price(merged_data["tick"][0]), decimal0=6, decimal1=18)))
merged_data["position_value"] = (token0_amount  + merged_data["fee0_sum"]) + (token1_amount + merged_data["fee1_sum"]) * merged_data["real_price"]
merged_data["position_value0"] = (token0_amount  + merged_data["fee0_sum"])
merged_data["position_value1"] = (token1_amount + merged_data["fee1_sum"]) * merged_data["real_price"]
volatility = metric.volatility(merged_data["position_value"])
roi = metric.roi(merged_data["position_value"][0], merged_data["position_value"].iloc[-1], (end - begin).days)
print("** volatility = ", volatility)

rf = 0.0001 # risk free rate 先随便写一个
sharpe_ratio = metric.sharpRatio(roi, volatility, rf)

** volatility =  1.8190218863695696e-12
